# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [2]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [3]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/06 11:41:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [4]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [5]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows


In [6]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [7]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   NULL|     BE|   NULL|    NULL|      1|  NULL|   269|  6|    69| NULL|       1|    NULL|    0.0|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|3070802| 1963| 1096|   NULL|     US|     TX|    NULL|      1|  NULL|     2|  6|    63| NULL|       0|    NULL|   NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|3070803| 1963| 1096|   NULL|     US|     IL|    NULL|      1|  NULL|     2|  6|    6

Double left join citations to patent table to get citing and cited state. Left join ensures all rows in citing table. Finally I drop null values filtering out lagging null patents and states.

In [23]:
citations_with_states = citations.join(patents.select("PATENT", "POSTATE"), citations.CITING == patents.PATENT, how='left')\
            .withColumnRenamed("POSTATE", "CITING_STATE").withColumnRenamed("PATENT", "CITING_PATENT").alias("citing_state")\
            .join(patents.alias("patents2").select("PATENT", "POSTATE"), col("citing_state.CITED") == col("patents2.PATENT"), how='left')\
            .withColumnRenamed("POSTATE", "CITED_STATE").withColumnRenamed("PATENT", "CITED_PATENT")\
            .na.drop()\
            .drop("CITING_PATENT", "CITED_PATENT")


Filter to get same state citations. Group by citing patent and count citations. Join with patents to get patent names. Order by citation count and show.

In [28]:
citations_with_states.filter(col("CITING_STATE") == col("CITED_STATE"))\
            .groupBy("CITING")\
            .agg(count("CITING").alias("CITATION_COUNT"))\
            .join(patents, col("CITING") == col("PATENT"), how='inner')\
            .orderBy(col("CITATION_COUNT").desc())\
            .show()



+-------+--------------+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| CITING|CITATION_COUNT| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+--------------+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|5959466|           125|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  NULL|   326|  4|    46|  159|       0|     1.0|   NULL|  0.6186|    NULL|  4.8868|  0.0455|   0.044|    NULL|    NULL|
|5983822|           103|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  NULL|   114|  5|    55|  200|       0|   0.995|   NULL|  0.7201|    NULL|   12.45|     0.